In [2]:
import pandas as pd
import numpy as np

from scipy.spatial import distance
from scipy.optimize import minimize
from rdkit import Chem
from rdkit.Chem import AllChem

import joblib

from reader_writer import xyz_to_df

In [3]:
df = xyz_to_df('ethanol.xyz')
r_c = joblib.load('r_c.pkl')
r_c
df['atomnum'] = np.arange(1, df.shape[0]+1)
df

,atom,x,y,z,atomnum
0,H,-2.080143,0.432973,0.072282,1
1,C,-1.212970,-0.229529,-0.009716,2
2,H,-1.265591,-0.953986,0.809795,3
3,C,0.084976,0.559039,0.051055,4
4,O,1.232231,-0.273190,-0.127612,5
5,H,0.150614,1.120025,0.994302,6
6,H,1.247388,-0.899874,0.615068,7
7,H,0.131609,1.284181,-0.764522,8
8,H,-1.273754,-0.774863,-0.954059,9


In [38]:
class Molecule:
    def __init__(self, df, r_c, k_E_r=100):
        self.df = df
        self.r_c = r_c
        self.create_bond_matrix()
        self.create_distance_mask()
        self.k_E_r = k_E_r
        self.create_pdb_string()
        self.create_rdkitmol()
        self.new_df()
        self.conj_matrix()
    
    def new_df(self):
        conf = self.rdkit_mol.GetConformer()
        
        self.rdkit_df = pd.DataFrame([[atom.GetSymbol(), conf.GetAtomPosition(cnt).x, conf.GetAtomPosition(cnt).y, conf.GetAtomPosition(cnt).z,
        atom.GetProp("_GasteigerCharge")] for cnt, atom in enumerate(ethanol.rdkit_mol.GetAtoms())], columns = ['atom', 'x', 'y', 'z', 'charge'])

    def conj_matrix(self):
        n_atoms = mol.GetNumAtoms()

        # Создайте пустую матрицу связности
        adj_matrix = np.zeros((n_atoms, n_atoms))

        # Заполните верхний треугольник матрицы связности
        for atom in mol.GetAtoms():
            for neighbor in atom.GetNeighbors():
                idx1 = atom.GetIdx()
                idx2 = neighbor.GetIdx()
                if idx1 < idx2:  # только для верхнего треугольника
                    adj_matrix[idx1, idx2] = 1

        self.adj_matrix = adj_matrix

    def create_pdb_string(self):
        self.pdb_string = '\n'.join(self.df.apply(lambda x: 'ATOM  {:5d}  {:^4s}MOL     1    {:8.3f}{:8.3f}{:8.3f}  1.00  0.00          {:>2s} '
                 .format(x['atomnum'], x['atom'], x['x'], x['y'], x['z'], x['atom']), axis=1).values)
        
    def create_rdkitmol(self):
        self.rdkit_mol = Chem.MolFromPDBBlock(self.pdb_string, removeHs=False)

    def calc_charge(self):
        AllChem.ComputeGasteigerCharges(self.rdkit_mol)


    def create_bond_matrix(self, tol=0.2):
        coords = self.df[['x', 'y', 'z']].values
        atoms = self.df['atom'].values

        dist_matrix = distance.cdist(coords, coords, 'euclidean')
        bond_matrix = np.zeros(dist_matrix.shape, dtype=bool)

        n = len(atoms)
        for i in range(n):
            for j in range(i+1, n):  # Only need to consider half the matrix
                atom_i = atoms[i].lower()
                atom_j = atoms[j].lower()

                # Ensure we have bond length data for these atoms
                if atom_i in self.r_c and atom_j in self.r_c[atom_i]:
                    if abs(dist_matrix[i, j] - self.r_c[atom_i][atom_j]) < tol:
                        bond_matrix[i, j] = bond_matrix[j, i] = True  # Atoms are likely bonded
        self.df_bonds = pd.DataFrame(bond_matrix, index=atoms, columns=atoms)

    def create_distance_mask(self):
        atoms = self.df['atom'].values
        n = len(atoms)

        dist_mask = np.zeros((n, n), dtype=float)

        for i in range(n):
            for j in range(i+1, n):  # Only need to consider half the matrix
                atom_i = atoms[i].lower()
                atom_j = atoms[j].lower()

                # Ensure we have bond length data for these atoms
                if atom_i in self.r_c and atom_j in self.r_c[atom_i]:
                    dist_mask[i, j] = dist_mask[j, i] = self.r_c[atom_i][atom_j]

        self.df_dist_mask = pd.DataFrame(dist_mask, index=atoms, columns=atoms)
    
    def energy_distance(self, coords):
        # coords = self.df[['x', 'y', 'z']].values
        coords = coords.reshape(df[['x', 'y', 'z']].shape)
        dist_matrix = distance.cdist(coords, coords, 'euclidean')
        return np.sum(self.k_E_r * (dist_matrix - self.df_dist_mask.values) ** 2 \
                      * self.df_bonds.values) / 2


ethanol = Molecule(df, r_c)
ethanol.calc_charge()
ethanol.rdkit_mol
ethanol.rdkit_df

,atom,x,y,z,charge
0,H,-2.080,0.433,0.072,0.025373291592356649
1,C,-1.213,-0.230,-0.010,-0.041838486154772384
2,H,-1.266,-0.954,0.810,0.025373291592356649
3,C,0.085,0.559,0.051,0.040220581749086423
4,O,1.232,-0.273,-0.128,-0.39666371478125978
5,H,0.151,1.120,0.994,0.056069718921006324
6,H,1.247,-0.900,0.615,0.21002230656786322
7,H,0.132,1.284,-0.765,0.056069718921006324
8,H,-1.274,-0.775,-0.954,0.025373291592356649


In [38]:
def find_minima(f, initial_guess):
    result = minimize(f, initial_guess, method='trust-constr')
    if result.success:
        return result.x, result.fun
    else:
        raise Exception("The optimization process was not successful.")
    
find_minima(ethanol.energy_distance, ethanol.df[['x', 'y', 'z']].values.flatten())

(array([-2.07873172,  0.4328589 ,  0.07216627, -1.2203252 , -0.23396007,
        -0.00901677, -1.26462523, -0.95287648,  0.80908878,  0.09534658,
         0.56396543,  0.05361414,  1.22833655, -0.27287322, -0.12648248,
         0.14876096,  1.11753864,  0.99105939,  1.2473821 , -0.8990527 ,
         0.61408252,  0.13183892,  1.28388441, -0.76399516, -1.27362396,
        -0.7747082 , -0.95392455]),
 2.1578724683049923e-14)